In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from math import log, sqrt

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchsde
from src.energies.base_prior import MeanFreePrior, Prior
from src.energies.gmm_energy import GMM, GMMTempWrapper
from src.models.components.clipper import Clipper
from src.models.components.mlp import MyMLP, MyMLPTemperature
from src.utils.data_utils import remove_mean
from torch import vmap
from torch.func import hessian
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torchdiffeq import odeint
from tqdm import tqdm

from fab.target_distributions import gmm

In [ ]:
import glob
from collections import defaultdict

import numpy as np

noise_std = 0.2

samples = defaultdict(list)

In [ ]:
n_particles = 13
spatial_dim = 3
dimensionality = spatial_dim * n_particles

In [ ]:
temps = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.5, 2.0, 3.0, 4.0]
for i in range(1, 121):
    temp_idx = (i - 1) // 10
    temp = temps[temp_idx]
    path_pattern = f"/network/scratch/a/alexander.tong/lj13_samples/samples_v22_{i}.npy"

    # try catch permission error

    files = sorted(glob.glob(path_pattern))
    # print(f"Found {len(files)} files for temperature {temp}")
    arrays = []
    for file in files:
        try:
            arrays.append(np.load(file))
        except PermissionError:
            print(f"Permission error for file {file}")
            break

    s = np.concatenate(arrays, axis=0)
    s = s.reshape(-1, dimensionality)
    samples[temp].append(s)

for temp in temps:
    print(temp)
    if len(samples[temp]) == 0:
        continue
    samples[temp] = np.concatenate(samples[temp], axis=0)
    print(f"Samples at {temp} has shape: {samples[temp].shape}")

    idxs = np.random.permutation(samples[temp].shape[0])
    train_idx = idxs[:10000]
    val_idx = idxs[10000:20000]
    test_idx = idxs[20000:30000]

    samples_train = samples[temp][train_idx]
    samples_val = samples[temp][val_idx]
    samples_test = samples[temp][test_idx]

    # if directory does not exist, create it
    import os

    temp = temp.__float__()
    data_path = f"../../data/LJ13_temp_{temp}"
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    else:
        print(f"Directory {data_path} already exists. Overwriting data.")

    np.save(data_path + f"/train_split_LJ13-1000.npy", samples_train)
    np.save(data_path + f"/val_split_LJ13-1000.npy", samples_val)
    np.save(data_path + f"/test_split_LJ13-1000.npy", samples_test)
    print(f"Saved data for temperature {temp} to {data_path}")

In [ ]:
temps = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.5, 2.0, 3.0, 4.0]
for temp in temps:
    path_pattern = f"/network/scratch/a/alexander.tong/lj13_samples/samples_v23_{temp}*"
    # try catch permission error

    files = sorted(glob.glob(path_pattern))
    print(f"Found {len(files)} files for temperature {temp}")
    arrays = []
    for file in files:
        try:
            arrays.append(np.load(file))
        except PermissionError:
            print(f"Permission error for file {file}")
            break
    if len(arrays) == 0:
        print(f"No files found for temperature {temp}")
        continue

    samples = np.concatenate(arrays)
    print(f"Concatenated array shape: {samples.shape}")
    samples = samples.reshape(-1, dimensionality)

    idxs = np.random.permutation(samples.shape[0])
    train_idx = idxs[:10000]
    val_idx = idxs[10000:20000]
    test_idx = idxs[20000:30000]

    samples_train = samples[train_idx]
    samples_val = samples[val_idx]
    samples_test = samples[test_idx]

    # if directory does not exist, create it
    import os

    temp = temp.__float__()
    data_path = f"../../data/LJ13_temp_{temp}"
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    else:
        print(f"Directory {data_path} already exists. Overwriting data.")

    np.save(data_path + f"/train_split_LJ13-1000.npy", samples_train)
    np.save(data_path + f"/val_split_LJ13-1000.npy", samples_val)
    np.save(data_path + f"/test_split_LJ13-1000.npy", samples_test)
    print(f"Saved data for temperature {temp} to {data_path}")